In [2]:
from ibm_watson import SpeechToTextV1, ApiException, NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import BasicAuthenticator
import json, os

In [3]:
# Basic Authentication with Watson STT API
stt_authenticator = BasicAuthenticator(
   'apikey',
   'qa8K2ibwTd2CoJAvrUfhSNaZL2nlrIjaafpjTxslF30C'
)

# Construct a Watson STT client with the authentication object
stt = SpeechToTextV1(
    authenticator=stt_authenticator
)

# Set the URL endpoint for your Watson STT client
stt.set_service_url(
    'https://api.us-south.speech-to-text.watson.cloud.ibm.com'
)

# Read audio file and call Watson STT API:
with open(
    'audio_sample.flac', 'rb'
) as audio_file:
    # Transcribe the audio.flac with Watson STT
    # Recognize method API reference: 
    # https://cloud.ibm.com/apidocs/speech-to-text?code=python#recognize
    stt_result = stt.recognize(
        audio=audio_file,
        content_type='audio/flac',
        model='pt-BR_BroadbandModel'
    ).get_result()

# Print STT API call results
print(json.dumps(stt_result, indent=2))

{
  "result_index": 0,
  "results": [
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "esse carro espetacular visual ind\u00edcio com bela cabalmente que m\u00eddia compat\u00edvel com a droga de alto e a colocar entretanto o consumo deixa a desejar fa\u00e7o de sete a nove quil\u00f4metros livre na cidade com gasolina ",
          "confidence": 0.86
        }
      ]
    }
  ]
}


In [23]:
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions

# Basic Authentication with Watson NLU API
nlu_authenticator = BasicAuthenticator(
    'apikey',
    'skEXm4q1Mn9XLLuQBOawyrovnjER9QDMLyht__-F4QtT'
)

# Construct a Watson NLU client with the authentication object
nlu = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=nlu_authenticator
)

# Set the URL endpoint for your Watson STT client
nlu.set_service_url(
    'https://api.us-south.natural-language-understanding.watson.cloud.ibm.com'
)

texto = "O Argo tem um ar-condicionado não muito eficiente e o motor também é muito ruim. Os bancos são confortávies"


# call Watson API
nlu_result = nlu.analyze(
        text=texto,
        features=Features(entities=EntitiesOptions(sentiment=True,limit=5, model='ab9e03c9-ea05-44db-9466-5a256f6d7c3b'))
    ).get_result()


# Print NLU API call results
print(json.dumps(nlu_result, indent=2))


{
  "usage": {
    "text_units": 1,
    "text_characters": 107,
    "features": 1
  },
  "language": "pt",
  "entities": [
    {
      "type": "CONFORTO",
      "text": "bancos",
      "sentiment": {
        "score": 0,
        "label": "neutral"
      },
      "disambiguation": {
        "subtype": [
          "NONE"
        ]
      },
      "count": 1,
      "confidence": 0.999637
    },
    {
      "type": "MODELO",
      "text": "Argo",
      "sentiment": {
        "score": -0.676244,
        "label": "negative"
      },
      "disambiguation": {
        "subtype": [
          "NONE"
        ]
      },
      "count": 1,
      "confidence": 0.999395
    },
    {
      "type": "DESEMPENHO",
      "text": "motor",
      "sentiment": {
        "score": -0.676244,
        "label": "negative"
      },
      "disambiguation": {
        "subtype": [
          "NONE"
        ]
      },
      "count": 1,
      "confidence": 0.996413
    },
    {
      "type": "ACESSORIOS",
      "text": "ar-

In [90]:
entidades = nlu_result['entities']

# primeiro criar um set, com as entidades e modelos

rank_entidades = dict()
result_entidades = []
modelo = ''

for entidade in entidades:
    ent = entidade['type']
    sent = entidade['sentiment']['score']
    mention = entidade['text']
    if (ent != 'MODELO'):
        if ent in rank_entidades:
            if sent < rank_entidades[ent]:
                rank_entidades[ent] = sent
        else:
            rank_entidades[ent] = sent
    else:
        modelo = mention
            
    result_entidades.append({'entity': ent, 
                            'sentiment': sent, 
                           'mention': mention})

print(modelo)
print(rank_entidades)
print(result_entidades)

Argo
{'CONFORTO': 0, 'DESEMPENHO': -0.676244, 'ACESSORIOS': -0.676244}
[{'entity': 'CONFORTO', 'sentiment': 0, 'mention': 'bancos'}, {'entity': 'MODELO', 'sentiment': -0.676244, 'mention': 'Argo'}, {'entity': 'DESEMPENHO', 'sentiment': -0.676244, 'mention': 'motor'}, {'entity': 'ACESSORIOS', 'sentiment': -0.676244, 'mention': 'ar-condicionado'}]


In [60]:
prioridades = {
    'SEGURANCA': 1,
    'CONSUMO': 2,
    'DESEMPENHO': 3,
    'MANUTENCAO': 4,
    'CONFORTO': 5,
    'DESIGN': 6,
    'ACESSORIOS': 7
              }

In [62]:
sugestoes = {
    'SEGURANCA': ['FIAT 500','DUCATO', 'RENEGADE'],
    'CONSUMO': ['ARGO', 'TORO', 'DUCATO'],
    'DESEMPENHO': ['ARGO', 'CRONOS', 'MAREA'],
    'MANUTENCAO': ['FIORINO', 'CRONOS', 'ARGO'],
    'CONFORTO': ['ARGO', 'CRONOS', 'TORO'],
    'DESIGN': ['FIAT 500', 'ARGO', 'RENEGADE'],
    'ACESSORIOS': ['RENEGADE', 'ARGO', 'TORO']    
}

In [86]:
menor = rank_entidades[min(rank_entidades, key=rank_entidades.get)]
print(menor)

-0.676244


In [105]:
#rank_entidades['CONSUMO'] = -0.576243

In [106]:
#most_negatives = {k: v for k, v in rank_entidades.items() if v == min}
most_negatives = {k: prioridades[k] for k, v in rank_entidades.items() if abs(v - menor) < 0.1}
most_negatives

{'DESEMPENHO': 3, 'ACESSORIOS': 7}

In [84]:
recomendado = sugestoes[min(most_negatives, key = most_negatives.get)]
recomendado

['ARGO', 'CRONOS', 'MAREA']

In [75]:
retorno = {
    'recommendation': '',
}

In [80]:
retorno['recommendation'] = recomendado[0]
retorno['entities'] = result_entidades

retorno

{'recommendation': 'ARGO',
 'entities': [{'entity': 'CONFORTO', 'sentiment': 0, 'mention': 'bancos'},
  {'entity': 'MODELO', 'sentiment': -0.68, 'mention': 'Argo'},
  {'entity': 'DESEMPENHO', 'sentiment': -0.68, 'mention': 'motor'},
  {'entity': 'ACESSORIOS', 'sentiment': -0.68, 'mention': 'ar-condicionado'}]}